In [ ]:
import os.path
import pandas as pd

import fitz

import string
translation_table = str.maketrans('', '', string.digits)

import PyPDF2

from lingua import Language, LanguageDetectorBuilder
languages = [Language.ENGLISH, Language.FRENCH, Language.GERMAN, Language.SPANISH, Language.CHINESE, Language.TAGALOG, Language.SOTHO, Language.YORUBA]
detector = LanguageDetectorBuilder.from_languages(*languages).build()
#detector = LanguageDetectorBuilder.from_all_languages().with_preloaded_language_models().build()

In [ ]:
if (os.path.exists("data/langmeta.csv")):
    os.remove("data/langmeta.csv")

In [ ]:
pdfurl = pd.read_csv("data/downloadedpdf.csv").fillna(value = "")

In [ ]:
text= ""
for index, row in pdfurl.iterrows():    
    unique = row['URL'].removeprefix("https://bildungsportal.sachsen.de/opal/oer/")
    unique = unique.removesuffix("\n")
    metadict = {"Identifier":unique}   
    pfad = "temp/" + unique + ".pdf"
#Falls beim Download kein Fehler aufgetreten ist: Probier die Datei zu öffnen
    if((row['Error'] == "")):
        #saved as .txt if .txt doesn't exist yet
        textpfad = "temp/" + unique + ".txt"
        if (not os.path.exists(textpfad)):
            try:
                pdfFileObj = open(pfad, 'rb')
                pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
                for x in range(0,pdfReader.numPages):        
                    pageObj = pdfReader.getPage(x)        
                    text = text + pageObj.extractText()    
            except:
                print("An Error occured while reading.")
                frame = pd.DataFrame(metadict, [0])
                if (os.path.exists("data/langmeta.csv")):   
                    #oldframe = pd.read_csv("data/langmeta.csv")
                    newframe = pd.concat([oldframe, frame])
                    newframe.to_csv("data/langmeta.csv", index=False)
                    oldframe = newframe
                else:
                    frame.to_csv("data/langmeta.csv", index=False)
                    oldframe = frame
                metadict.clear
                continue
            #save
            with open(textpfad, 'w', encoding="utf-8") as textfile:
                textfile.write(text.encode('utf-8', 'replace').decode())
        else:
            with open(textpfad, 'r', encoding="utf-8") as textfile:
                text = textfile.read()
    #Größe bestimmen
        metadict.update({"Textsize":len(text)})
    #Titel bestimmen
        with fitz.open(pfad) as pdffile:
            header = ""  
            header_size = 0.0
            first_span = True  
   
            for page in pdffile:
                txtpage = page.get_textpage()
                DICT = txtpage.extractDICT()
                for block in DICT["blocks"]:
                    if block['type'] == 0:
                        for line in block["lines"]:
                            for span in line["spans"]:
                                if(first_span == True):
                                    header = span["text"]
                                    header_size = span["size"]
                                    first_span = False
                                elif(span["size"] > header_size):
                                    header = span["text"]
                                    header_size = span["size"]    
                break
            metadict.update({"texttitle":header})
    #Headerextraction
        headertextpfad = "temp/" + unique + "_header.txt"
        if (not os.path.exists(headertextpfad)):
        #Die verschiedenen Schiftgrößen bestimmen
            with fitz.open(pfad) as pdffile:
                sizedict = {0:0}
                fullsize = 0
                for page in pdffile:
                    txtpage = page.get_textpage()
                    DICT = txtpage.extractDICT()
                    for block in DICT["blocks"]:
                        if block['type'] == 0:
                            for line in block["lines"]:
                                for span in line["spans"]:
                                    if(int(span["size"]) in sizedict):
                                        sizedict[int(span["size"])] += len(span["text"])
                                    else:
                                        sizedict[int(span["size"])]= len(span["text"])
                                    fullsize += len(span["text"])
                                
            #Welche Größen sind Header?
            if(fullsize > 0):
                headersize = 0
                sortsizedict = sorted(sizedict, reverse=True)
                i_max = 0
                for key in sortsizedict:
                    i = i_max -1
                    if(i>=0):
                        sizedict[sortsizedict[i_max]]+= sizedict[sortsizedict[i]]
                    if(sizedict[sortsizedict[i_max]] > fullsize/4):
                        i_max -= 1
                        break
                    i_max += 1
            
                if(i_max < 0):
                    i_max = 0
                headersize = sortsizedict[i_max]
        
            #header auslesen
                headertext=""
                with fitz.open(pfad) as pdffile:
                    for page in pdffile:
                        txtpage = page.get_textpage()
                        DICT = txtpage.extractDICT()
                        for block in DICT["blocks"]:
                            if block['type'] == 0:
                                for line in block["lines"]:
                                    for span in line["spans"]:
                                        if(int(span["size"]) >= headersize):
                                            headertext += span["text"].translate(translation_table) +" "
                with open(headertextpfad, 'w', encoding="utf-8") as textfile:
                    textfile.write(headertext.encode('utf-8', 'replace').decode())   
                                
    #Spracherkennung
        if(len(text)>=100):            
            language = detector.detect_language_of(text) #compute_language_confidence_values(text)
            if(language is not None):
                metadict.update({"Language":language.name})
        frame = pd.DataFrame(metadict, [0])
        if (os.path.exists("data/langmeta.csv")):   
            #oldframe = pd.read_csv("data/langmeta.csv")
            newframe = pd.concat([oldframe, frame])
            newframe.to_csv("data/langmeta.csv", index=False)
            oldframe = newframe
        else:
            frame.to_csv("data/langmeta.csv", index=False)
            oldframe = frame
        metadict.clear
        text=""
        print("Finished: ", unique)